# Ejercicio de análisis e interpretación de datos
## Oscar Estrada - Alianza Bioversity CIAT

### APRENDIZAJE SUPERVISADO

### 1. Cargar librerías

In [67]:
# Librerias necesarias para la manipulacion de datos, exploracion y modelacion
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid") #whitegrid
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import PartialDependenceDisplay
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")

### 2. Cargar datos

In [68]:
# Cargar datos desde GitHub
df = pd.read_csv('https://github.com/oestradavargas/Taller_Analitica_datos/raw/main/datos.csv', encoding='latin')

In [ ]:
# Ver tamaño de la base de datos (filas, columnas)
df.shape

In [ ]:
# Ver el encabezado de la base de datos
df.head()

In [ ]:
# Identificacion de variables numericas
var_numericas =  (df.drop('REND', axis=1)).select_dtypes('number').columns
var_numericas

In [72]:
# Codificacion de variables dicotomicas SI/NO
#pd.set_option('future.no_silent_downcasting', True)
df = df.replace({'Si': 1, 'No': 0})

In [ ]:
df.head()

In [ ]:
# Ver cantidad de datos y tipo de variables
df.info()

### 3.1. Análisis exploratorio (numérico)

In [ ]:
# Identificar las variables con datos faltantes NA
df.isna().sum(0).sort_values(ascending=False).head()

In [76]:
# Eliminar registros con datos faltantes
df = df.dropna()

In [ ]:
df.isna().sum(0).sort_values(ascending=False).head()

In [ ]:
df.shape

In [ ]:
# Resumen estadistico de las variables numericas
df.describe()

In [ ]:
# Descripcion de las variables categoricas
df.describe(include='object')

In [ ]:
# Exploracion de una variable categorica individualmente
df['TEXT'].value_counts()

In [82]:
# Eliminar una variable
df = df.drop('TEXT', axis=1)

### 3.2 Análisis exploratorio (gráfico)

In [ ]:
# Histograma de la libreria Seaborn
plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='REND', bins=18, kde=True)
plt.title('HISTOGRAMA DE RENDIMIENTO', fontsize=16, fontweight='bold')
plt.xlabel('Rendimiento (ton/ha)')
plt.ylabel('Frecuencia')
plt.tight_layout()
plt.show()

In [ ]:
# Boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='REND')
plt.title('BOXPLOT DE RENDIMIENTO', fontsize=16, fontweight='bold')
plt.xlabel('(ton/ha)')
plt.tight_layout()
plt.show()

In [ ]:
# Boxplots de fertilizantes
fig, axes = plt.subplots(1, 3, figsize=(10, 6))
sns.boxplot(y='N_TOTAL', data=df, color = '#1f77b4', ax=axes[0])
axes[0].set_title('N TOTAL', fontweight = 'bold')
axes[0].set_ylabel('(kg/ha)')
sns.boxplot(y='P2O5_TOTAL', data=df, color = '#1f77b4', ax=axes[1])
axes[1].set_title('P2O5 TOTAL', fontweight = 'bold')
axes[1].set_ylabel('(kg/ha)')
sns.boxplot(y='K2O_TOTAL', data=df, color = '#1f77b4', ax=axes[2])
axes[2].set_title('K2O TOTAL', fontweight = 'bold')
axes[2].set_ylabel('(kg/ha)')
fig.suptitle('BOXPLOTS DE FERTILIZANTES QUIMICOS TOTALES', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Grafico de dispersión
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, y='REND', x='N_TOTAL')
plt.title('N_TOTAL vs RENDIMIENTO', fontsize=16, fontweight='bold')
plt.xlabel('N_TOTAL (kg/ha)')
plt.ylabel('Rendimiento (ton/ha)')
plt.tight_layout()
plt.show()

In [87]:
# Eliminar valores fuera de rango para los fertilizantes
df = df[(df['N_TOTAL'] >= 40) & (df['N_TOTAL'] <= 350)]
df = df[(df['P2O5_TOTAL'] >= 20) & (df['P2O5_TOTAL'] <= 150)]
df = df[(df['K2O_TOTAL'] >= 20) & (df['K2O_TOTAL'] <= 200)]

In [ ]:
df.shape

In [ ]:
# Grafico de frecuencias de métodos de siembra
plt.figure(figsize=(10, 6))
ax = sns.countplot(x="MET_SIEMBRA", data=df)

# Agregar etiquetas de frecuencia en cada barra
for p in ax.patches:
    ax.annotate(f'{p.get_height()} ({p.get_height() / len(df) * 100:.2f}%)',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points', fontsize=12)

plt.title('METODO DE SIEMBRA', fontsize=16, fontweight='bold')
plt.xlabel('Método')
plt.ylabel('Frecuencia')
plt.tight_layout()
plt.show()

### 4. Modelacion (Random Forest)

In [90]:
# Codificar las variables categoricas
df = pd.get_dummies(df, drop_first=True, dtype = int)
df = df.reset_index(drop=True)

In [ ]:
df.shape

In [91]:
#### Definicion de X y Y
X, y = df.drop(columns="REND"), df["REND"]

In [92]:
### Particionamiento de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state= 42)

In [ ]:
print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)
print('X_test: ', X_test.shape)
print('y_test: ', y_test.shape)

In [ ]:
# Identificacion de las variables categoricas
var_categoricas = list(set(X_train.columns) - set(var_numericas))
var_categoricas = pd.Index(var_categoricas)
var_categoricas

In [ ]:
# Modelo RandomForest
modelo = RandomForestRegressor(random_state=42,
                               n_jobs=-1)

modelo.fit(X_train, y_train)

In [ ]:
# Metricas de evaluacion (R2)
train_score = modelo.score(X_train, y_train)*100
print(f'R2 de entrenamiento: {train_score:.3f} %')
test_score = modelo.score(X_test, y_test)*100
print(f'R2 de prueba: {test_score:.3f} %')

### 4.1 Optimización del modelo

In [ ]:
# Define el modelo a optimizar
modelo = RandomForestRegressor(random_state=42, n_jobs=-1)

# Define el espacio de búsqueda de los hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': (2, 5, 10),
    'max_features': [0.2, 0.5, 0.7],
    'max_samples': [0.1, 0.6, 0.7],
    'max_leaf_nodes': (2, 5, 10)
}

# Configura GridSearchCV
grid_search = GridSearchCV(estimator=modelo,
                           param_grid=param_grid,
                           scoring='neg_root_mean_squared_error',
                           cv=3,  # Número de pliegues para la validación cruzada
                           n_jobs=-1)

# Ajusta el modelo
#grid_search.fit(X_train, y_train)
grid_search.fit(X_train, y_train)

# Muestra el mejor conjunto de parámetros
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor RMSE:", -grid_search.best_score_)

# Obtener el mejor modelo
modelo = grid_search.best_estimator_

In [ ]:
# Metricas de evaluacion (R2)
train_score = modelo.score(X_train, y_train)*100
print(f'R2 de entrenamiento: {train_score:.3f} %')
test_score = modelo.score(X_test, y_test)*100
print(f'R2 de prueba: {test_score:.3f} %')

### 5.1 Interpretación: Gráfico de relevancia de variables

In [ ]:
# Grafico de relevancia de variables
n_perf=20

nombres_var = modelo.feature_names_in_
data_model = pd.concat([pd.Series(nombres_var),
                        pd.Series(modelo.feature_importances_ )], axis=1)
data_model.columns = ['Feature', 'Values']
data_model = data_model.sort_values('Values', ascending=False)
data_model = data_model.iloc[0:n_perf]

plt.figure(figsize=(10, 6))
sns.barplot(data=data_model, x='Values', y='Feature', color='#1f77b4')
plt.title('Relevancia de Variables', size=16, fontweight='bold')
plt.xticks(rotation=90)
plt.xlabel('Relevancia media')
plt.ylabel('Variable')
plt.tight_layout()
plt.show()

### 5.2 Interpretación: Gráfico de dependencia parcial

In [ ]:
# Grafico de dependencia parcial de las variables
variables = ['TEMP_MIN_PROM']

common_params = {
    "subsample": 0.999,
    "n_jobs": 2,
    "grid_resolution": 10,
    "random_state": 42
}

features_info = {
    # features of interest
    "features": variables,
    # type of partial dependence plot: average, individual, both
    "kind": "average",
    # information regarding categorical features
    "categorical_features": var_categoricas
}

display = PartialDependenceDisplay.from_estimator(
    modelo,
    X_train,
    **features_info,
    **common_params,
)

### APRENDIZAJE NO SUPERVISADO

In [ ]:
# Cargar el dataset
data = pd.read_csv('https://github.com/oestradavargas/Taller_Analitica_datos/raw/main/datos.csv', encoding='latin')

# Seleccion de variables numéricas
data = data[data.select_dtypes('number').columns]

data['TEMP_PROM'] = (data['TEMP_MIN_PROM'] + data['TEMP_MAX_PROM']) / 2

# Seleccionar las variables de clima y suelo para el clustering
X = data[['TEMP_PROM', 'PREC_ACUM', 'REL_HUM_PROM', 'pH', 'C_ORG_SUELO']]

# Eliminar las filas con valores nulos
X = X.dropna()

# Determinar el número óptimo de clusters usando el método del codo
wcss = []  # within-cluster sum of squares (suma de cuadrados dentro de los clusters)
for i in range(1, 11):  # Probar con 1 a 10 clusters
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)  # inertia_ es el valor WCSS

# Gráfica del método del codo
plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), wcss, marker='o', linestyle='--')
plt.title('GRAFICA "METODO DEL CODO"', fontsize=16, fontweight='bold')
plt.xlabel('Número de Clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
# Basado en la gráfica, seleccionamos un número óptimo de clusters
optimal_clusters = 2
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
clusters = kmeans.fit_predict(X)

# Agregar los clusters al DataFrame original
data['Cluster'] = clusters

# Crear un gráfico de dispersión de las parcelas agrupadas
plt.figure(figsize=(10, 6))
sns.scatterplot(x='TEMP_PROM', y='PREC_ACUM', hue='Cluster', data=data, palette='Set1', s=100)
plt.title('CLUSTERING BASADO EN CLIMA Y SUELO', fontsize=16, fontweight='bold')
plt.xlabel('Temperatura media (°C)')
plt.ylabel('Precipitación acumulada (mm)')
plt.legend(title='Cluster')
plt.show()

# Visualizar las características promedio de cada cluster
cluster_means = data.groupby('Cluster').mean()
cluster_means.head()